In [ ]:
elastic_rods_dir = '../'
weaving_dir = '../../../Weaving/'
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt

In [ ]:
# Reloadable libraries
import finite_diff, importlib
importlib.reload(finite_diff)
from finite_diff import fd_hessian_test, fd_gradient_test, block_error, hessian_convergence_block_plot

In [ ]:
centerline_pos = np.array([[0, 0, 0], [1, 0, 0], [2.1, 0, 0], [3, 0, 0]])
deformed_centerline_pos = np.array([[0, 0, 0],[1, 0, 0],[1 + 0.6 * np.sqrt(0.5),  0.6 * np.sqrt(0.5), 0],[2 + 0.6 * np.sqrt(0.5),  0.6 * np.sqrt(0.5), 0]])
test_rod = elastic_rods.ElasticRod(centerline_pos)
test_rod.setDeformedConfiguration(deformed_centerline_pos, np.array([0, np.pi / 6.0, np.pi / 6.0]))
test_rod.setMaterial(elastic_rods.RodMaterial('ellipse', 200, 0.3, (0.1, 0.05), stiffAxis = elastic_rods.StiffAxis.D1))

In [ ]:
# The Hessian formulas are only accurate with an updated source frame.
test_rod.updateSourceFrame()

In [ ]:
variableDP = True
grad = test_rod.gradEnergyBend(variableDesignParameters = variableDP)

In [ ]:
test_rod.bendingEnergyType = elastic_rods.BendingEnergyType.Bergou2008

In [ ]:
theta_offset = test_rod.numVertices() * 3
rl_offset = theta_offset + test_rod.numEdges()
rk_offset = rl_offset + test_rod.numEdges()

In [ ]:
var_types = ['FULL', 'POS', 'THETA', 'RL', 'RK']
var_indices = {'FULL': range(0, test_rod.numExtendedDoF()),
                'POS': range(0, theta_offset),
              'THETA': range(theta_offset, rl_offset),
                 'RL': range(rl_offset, rk_offset),
                 'RK': range(rk_offset, test_rod.numExtendedDoF())}

In [ ]:
block_error(test_rod, var_indices, 'POS', 'RK', grad, eps=1e-6, variableDesignParameters=variableDP)

In [ ]:
block_error(test_rod, var_indices, 'THETA', 'RK', grad, eps=1e-7, variableDesignParameters=variableDP)

In [ ]:
block_error(test_rod, var_indices, 'RL', 'RK', grad, eps=1e-6, variableDesignParameters=variableDP)

In [ ]:
block_error(test_rod, var_indices, 'RK', 'RL', grad, eps=1e-6, variableDesignParameters=variableDP)

In [ ]:
direction = np.zeros(test_rod.numExtendedDoF())
direction[var_indices['RK']] = np.random.normal(0, 1, len(var_indices['RK']))

In [ ]:
def decompose_gradient(g):
    return {t: g[var_indices[t]] for t in var_types}

def log_hessian_error(fd_delta_gradient, analytic_delta_gradient):
    fd_components = decompose_gradient(fd_delta_gradient)
    an_components = decompose_gradient(analytic_delta_gradient)
    for k in fd_components:
        print(f'{k}\t{la.norm(fd_components[k] - an_components[k]) / la.norm(an_components[k])}')

stepSize = 1e-5
fd_delta_grad, an_delta_grad = fd_hessian_test(test_rod, stepSize, etype = elastic_rods.EnergyType.Bend, direction = direction, variableDesignParameters = variableDP)
log_hessian_error(fd_delta_grad, an_delta_grad)

In [ ]:
hessian_convergence_block_plot(test_rod, var_types, var_indices, variableDesignParameters=variableDP)